# This notebook

This notebook contains first bits and pieces of the yet to be developed model correlating climate/environmental factors with conflict occurrence.

## Import libraries and file with settings

In [13]:
import conflict_model

import pandas as pd
import geopandas as gpd
from configparser import RawConfigParser
import matplotlib.pyplot as plt
import numpy as np
import datetime
import netCDF4 as nc
import rasterstats as rstats
import xarray as xr
import rasterio as rio
import seaborn as sbs
from sklearn import svm, preprocessing, model_selection, metrics
import os, sys

In [2]:
conflict_model.utils.show_versions()

Python version: 3.7.7 (default, Apr 15 2020, 05:09:04) [MSC v.1916 64 bit (AMD64)]
conflict_model version: 0.0.1-beta
geopandas version: 0.7.0
xarray version: 0.15.1
rasterio version: 1.1.0
pandas version: 1.0.3
numpy version: 1.18.1
scikit-learn version: 0.22.1
matplotlib version: 3.2.1
seaborn version: 0.10.1
rasterstats version: 0.14.0


Geopandas versions lower than 0.7.0 do not yet have the clip function. The notebook will thus not work with these versions.

In [3]:
if gpd.__version__ < '0.7.0':
    sys.exit('please upgrade geopandas to version 0.7.0, your current version is {}'.format(gpd.__version__))

In this file all the settings for the analysis are defined. By 'parsing' it, all values are read for different sections. This is a simple way to make the code independent of the input data and settings.

In [4]:
settings_file = r'../data/run_setting.cfg'

In [5]:
config = RawConfigParser(allow_no_value=True)
config.read(settings_file)

['../data/run_setting.cfg']

In [6]:
#out_dir
out_dir = config.get('general','output_dir')
if not os.path.isdir(out_dir):
        os.makedirs(out_dir)
print('for the record, saving output to folder {}'.format(out_dir) + os.linesep)

for the record, saving output to folder C:\Users\hoch0001\Documents\_code\conflict_model\data\OUT



# Applying functions

In [7]:
gdf = conflict_model.utils.get_geodataframe(config)

reading csv file to dataframe C:\Users\hoch0001\Documents\_code\conflict_model\data\UCDP/ged191.csv
...DONE

translating to geopandas dataframe
...DONE



In [8]:
conflict_gdf, extent_gdf = conflict_model.selection.select(gdf, config)

filtering on conflict properties...
...filtering key best with lower value 5
...filtering key type_of_violence with value 1
...passing key country as it is empty
focussing on period between 2000 and 2011

reading extent and spatial aggregation level from file C:\Users\hoch0001\Documents\_code\conflict_model\data\waterProvinces/waterProvinces_Africa.shp
...DONE

clipping datasets to extent
...DONE

clipping to climate zones['BWh', 'BSh']
...DONE



# Functions

In [58]:
def rasterstats_totalEvap(gdf_in, config, sim_year, out_dir):

    print('calculating evaporation mean per aggregation unit')
    
    nc_fo = os.path.join(config.get('general', 'input_dir'), 
                         config.get('env_vars', 'evaporation'))

    nc_ds = xr.open_dataset(nc_fo)
    print(nc_ds)

    nc_var = nc_ds['total_evaporation']
    years = nc_ds['time'].values
    print(years)
    
    ds = nc.Dataset(nc_fo)
    time = ds.variables['time']
    dates = nc.num2date(time[:], time.units, only_use_cftime_datetimes=False)
    dates_pd = pd.to_datetime(dates)
    periods = dates_pd.to_period(freq='Y').strftime('%Y')
    
    print(periods)
    print(periods[0])
    
#     print(years)
#     year_int = []
#     for date in years:
#         date_object = datetime.datetime.strftime(datetime.datetime.utcfromtimestamp(date.astype('O')/1e9), format='%Y')
#         year_int = np.append(year_int, int(date_object))
#     print(year_int)
#     years = year_int.copy()
    
    years = years[years>=config.getint('settings', 'y_start')]
    years = years[years<=config.getint('settings', 'y_end')]

    affine = rio.open(nc_fo).transform

    gdf = gdf_in.copy()

    gdf['evap_mean_' + str(sim_year)] = np.nan

    nc_arr = nc_var.sel(time=sim_year)
    nc_arr_vals = nc_arr.values
    if nc_arr_vals.size == 0:
        raise ValueError('the data was found for this year in the nc-file {}, check if all is correct'.format(nc_fo))

    for i in range(len(gdf)):
        prov = gdf.iloc[i]
        zonal_stats = rstats.zonal_stats(prov.geometry, nc_arr_vals, affine=affine, stats="mean")
        gdf.loc[i, 'evap_mean_' + str(sim_year)] = zonal_stats[0]['mean']

    print('...DONE' + os.linesep)

    return gdf

# Analysis per year

In a first step, we want to know in which countries there was conflict or not. To that end, we first accumulate the number of fatalities per country and use this as proxy whether there was a conlfict or not (guess there is a rather strong like...).

In [59]:
print('simulation period from', str(config.getint('settings', 'y_start')), 'to', str(config.getint('settings', 'y_end')))
print('')

X1 = pd.DataFrame()
X2 = pd.DataFrame()
Y  = pd.DataFrame() # []

# go through all simulation years as specified in config-file
for sim_year in np.arange(config.getint('settings', 'y_start'), config.getint('settings', 'y_end'), 1):
    
    print('entering year {}'.format(sim_year) + os.linesep)
    
    # add column whether there was conflict/non-conflict in one year in one region
    out_df = conflict_model.analysis.conflict_in_year_bool(conflict_gdf, extent_gdf, config, sim_year, out_dir, saving_plots=True)
    
    # add column with zonal statistics of GDP per year per region
    out_df = conflict_model.env_vars_nc.rasterstats_GDP_PPP(out_df, config, sim_year, out_dir, saving_plots=True)
    
    out_df = rasterstats_totalEvap(out_df, config, sim_year, out_dir)
    
    # drop all rows with at least one MVs since sklearn does not like NaNs
    out_df = out_df.dropna()
    
    # create arrays with input variables X and target variable Y
    X1 = pd.concat([X1, out_df['GDP_PPP_mean_' + str(sim_year)]])
    X2 = pd.concat([X2, out_df['evap_mean_' + str(sim_year)]])
    Y = pd.concat([Y, out_df['boolean_conflict_' + str(sim_year)]])
    
    print('value points at end of this year:', len(X1), len(X2), len(Y))
        
    extent_gdf = out_df.copy() 
    
print('...simulation DONE')

simulation period from 2000 to 2011

entering year 2000

determining whether a conflict took place or not
...DONE

calculating GDP PPP mean per aggregation unit
[1990. 1991. 1992. 1993. 1994. 1995. 1996. 1997. 1998. 1999. 2000. 2001.
 2002. 2003. 2004. 2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012. 2013.
 2014. 2015.]


C:\Users\hoch0001\AppData\Local\Continuum\anaconda3\envs\conflict_model\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


...DONE

calculating evaporation mean per aggregation unit
<xarray.Dataset>
Dimensions:            (bnds: 2, latitude: 890, longitude: 850, time: 16)
Coordinates:
  * time               (time) datetime64[ns] 2000-07-15T12:00:00 ... 2015-07-15T12:00:00
  * longitude          (longitude) float32 -18.625 -18.541666 ... 52.125
  * latitude           (latitude) float32 38.291668 38.208332 ... -35.791668
Dimensions without coordinates: bnds
Data variables:
    time_bnds          (time, bnds) datetime64[ns] ...
    total_evaporation  (time, latitude, longitude) float32 ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    history:      Wed Jun 10 10:40:44 2020: cdo yearmean totalEvaporation_mon...
    institution:  Department of Physical Geography, Utrecht University
    Conventions:  CF-1.4
    description:  by Edwin H. Sutanudjaja (contact: e.h.sutanudjaja@uu.nl), P...
    title:        PCR-GLOBWB 2 output (not coupled to MODFLOW), with human fa.

TypeError: invalid type promotion

# Machine Learning

## Data preparation

First, create the X (variables) from all columns sampled and Y (target) and concatenate/copy.

In [ ]:
X_df = pd.concat([X1, 
                  X2], axis=1)

Y_df = Y.copy()

Then, convert them to numpy arrays

In [ ]:
X = X_df.to_numpy()

In [ ]:
Y = Y_df.to_numpy()

The scatterplot of the (two) variables in X looks like this. Also the sample size n is provided.

In [ ]:
plt.figure(figsize=(10,10))
sbs.scatterplot(x=X[:,0],
                y=X[:,1],  
                hue=Y[:,0])

plt.title('n=' + str(len(X1)))
plt.savefig(os.path.join(out_dir, 'scatter_plot.png'), dpi=300)
plt.show()

Before we can train and predict with the model, we need to scale the variable data and create trainings and test data for both variables and target.

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(preprocessing.scale(X),
                                                                    Y[:,0],
                                                                    test_size=0.5)

In [ ]:
preprocessing.scale(X).mean(axis=0), preprocessing.scale(X).std(axis=0)

## Model

### Train and predict

Create Support Vector Classification (SVC) model with balanced weight since data is unbalanced (e.g. many negative and few positive)

In [ ]:
clf = svm.SVC(class_weight='balanced')

Fit the model with the scaled training data and the boolean conflict data

In [ ]:
clf.fit(X_train, y_train)

Predict something with the scaled predition data

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
y_score = clf.decision_function(X_test)
y_score

### Evaluation

The **accuracy** is either the fraction (default) or the count (normalize=False) of correct predictions.

The **precision** is the ratio *tp / (tp + fp)* where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The **recall** is the ratio *tp / (tp + fn)* where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

In [ ]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))

**Precision-Recall** is a useful measure of success of prediction when the classes are very imbalanced. In information retrieval, precision is a measure of result relevancy, while recall is a measure of how many truly relevant results are returned.

The precision-recall curve shows the tradeoff between precision and recall for different threshold. A high area under the curve represents both high recall and high precision, where high precision relates to a low false positive rate, and high recall relates to a low false negative rate. High scores for both show that the classifier is returning accurate results (high precision), as well as returning a majority of all positive results (high recall).

A system with high recall but low precision returns many results, but most of its predicted labels are incorrect when compared to the training labels. A system with high precision but low recall is just the opposite, returning very few results, but most of its predicted labels are correct when compared to the training labels. An ideal system with high precision and high recall will return many results, with all results labeled correctly.

In [ ]:
average_precision = metrics.average_precision_score(y_test, y_score)

print('Average precision-recall score: {0:0.2f}'.format(average_precision))

In [ ]:
disp = metrics.plot_precision_recall_curve(clf, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: AP={0:0.2f}'.format(average_precision));

Results are pretty crappy, but that is okay give we use not very sensible input data at the moment...